# Colunas do dataset de teste:

 - semana (número inteiro): número da semana (1 a 5 de janeiro/2023)
 - pdv (número inteiro): código do ponto de venda
 - produto (número inteiro): código do SKU
 - quantidade (número inteiro): previsão de vendas

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install parquet -q
!pip install pyarrow -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import duckdb
import os
import pyarrow.parquet as pq
import pyarrow as pa # Import pyarrow core library

pd.set_option('display.max_columns', 60)
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.size'] = 24

from IPython.core.pylabtools import figsize

import seaborn as sns
sns.set(font_scale = 2)

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

# Modelo deve prever a quantidade com estas colunas no dataset df_teste, gerando a coluna quantidade

In [3]:
def missing_values_table(df):
        mis_val = df.isnull().sum()

        mis_val_percent = 100 * df.isnull().sum() / len(df)

        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

        print ("Seu dataframe tem " + str(df.shape[1]) + " colunas.\n"
            "Há " + str(mis_val_table_ren_columns.shape[0]) +
              " colunas que possuem valores ausentes.")

        return mis_val_table_ren_columns

In [4]:
def criar_features_para_treinamento(df, min_weeks=8):
    """
    Criar features com tratamento robusto de missing values
    """
    # Ensure 'quantidade' and 'data' columns are present before proceeding
    if 'quantidade' not in df.columns:
        print("Erro: Coluna 'quantidade' não encontrada no DataFrame.")
        return df
    if 'data' not in df.columns:
        print("Erro: Coluna 'data' não encontrada no DataFrame.")
        return df

    # Ensure 'data' is datetime type
    df['data'] = pd.to_datetime(df['data'])

    # Extract year and month from data
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month
    # Keep 'data' for sorting in other functions if needed, or remove here if not.
    # For now, let's keep it as other functions rely on it.

    # Create lags
    for lag in [1, 2, 3, 4, 52]:
        df[f'quantidade_lag_{lag}'] = df.groupby(['pdv', 'produto'])['quantidade'].shift(lag)

    # Moving averages
    for window in [4, 8, 13]:
        df[f'media_movel_{window}'] = df.groupby(['pdv', 'produto'])['quantidade'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )

    # Handle missing values in lags and moving averages
    # Let's apply a simple fillna(0) for demonstration after creation
    # A more sophisticated strategy might be needed based on the data
    lag_and_ma_cols = [col for col in df.columns if 'quantidade_lag_' in col or 'media_movel_' in col]
    for col in lag_and_ma_cols:
        df[col] = df[col].fillna(0)


    # Convert relevant columns to float32 to reduce memory usage
    # Convert 'quantidade' and the newly created lag/moving average features
    cols_to_convert_to_float32 = ['quantidade'] + lag_and_ma_cols
    for col in cols_to_convert_to_float32:
        if col in df.columns:
            df[col] = df[col].astype('float32')

    # Convert 'ano', 'mes', 'semana' to smaller integer types if possible
    for col in ['ano', 'mes', 'semana']:
        if col in df.columns:
            try:
                # Use pandas nullable integer type to handle potential NaNs if any crept in
                df[col] = df[col].astype('Int64') # Use Int64 or Int32 depending on range
            except Exception as e:
                print(f"Could not convert column '{col}' to Int64: {e}")
                # Fallback to standard int if no NaNs, otherwise keep original dtype or object
                try:
                     if df[col].isnull().sum() == 0:
                         df[col] = df[col].astype(int)
                except:
                     pass # Keep original dtype if conversion fails


    colunas = df.columns.tolist()
    print("Features:", colunas)
    return df

In [5]:
def criar_features_para_submissao(df_sub, df_treino, ano_previsao=2023):
    """
    Prepara o dataset de submissão (df_sub) adicionando features
    baseadas nos dados históricos (df_treino).
    Assume que df_sub já contém 'pdv', 'produto', e 'semana'.
    """
    df_sub_preparado = df_sub.copy()

    # Adicionar colunas temporais básicas (para 2023)
    df_sub_preparado['ano'] = ano_previsao
    df_sub_preparado['mes'] = 1  # Janeiro para semanas 1-5

    # To get the lag and moving average features for the submission data,
    # we need the *last* calculated features from the historical training data (df_treino).

    # Order historical data by date to easily get the last record for each pdv/produto
    # Ensure df_treino has 'data' column for sorting if it was removed elsewhere, or use another suitable column.
    # Assuming df_treino still has the 'data' column from previous steps.
    if 'data' in df_treino.columns:
        df_treino_sorted = df_treino.sort_values(['pdv', 'produto', 'data'])
    else:
        # If 'data' column is not available, sort by 'ano', 'mes', 'semana'
        df_treino_sorted = df_treino.sort_values(['pdv', 'produto', 'ano', 'mes', 'semana'])


    # Find the last record for each pdv/produto in the historical data (which contains calculated features)
    ultimo_registro_historico = df_treino_sorted.groupby(['pdv', 'produto']).tail(1).copy()

    # Select the feature columns from the last historical record
    # These are the features needed for prediction in the submission data
    features_to_merge = ['pdv', 'produto', 'quantidade_lag_1', 'quantidade_lag_2',
                         'quantidade_lag_3', 'quantidade_lag_4', 'quantidade_lag_52',
                         'media_movel_4', 'media_movel_8', 'media_movel_13']

    # Ensure that the features to merge actually exist in the historical data
    actual_features_to_merge = [col for col in features_to_merge if col in ultimo_registro_historico.columns]

    # Select only the actual features to merge from the last historical record
    ultimo_registro_historico_subset = ultimo_registro_historico[actual_features_to_merge]


    # Merge these historical features onto the submission dataset based on pdv and produto
    # Use left merge to keep all rows in the submission dataset
    df_sub_preparado = df_sub_preparado.merge(
        ultimo_registro_historico_subset,
        on=['pdv', 'produto'],
        how='left'
    )

    # Tratar lag_52 specifically for the correct week of the previous year (in df_treino)
    # We need the average of the same week from the previous year in the historical data
    # This part of the logic might be redundant if 'quantidade_lag_52' is already in ultimo_registro_historico_subset
    # based on how it was calculated in create_training_features.
    # Let's keep it for now but be mindful of potential issues.
    if 'semana' in df_sub_preparado.columns and 'semana' in df_treino.columns:
        semana_ano_anterior_stats = df_treino.groupby(['pdv', 'produto', 'semana'])['quantidade'].mean().reset_index()
        semana_ano_anterior_stats = semana_ano_anterior_stats.rename(columns={'quantidade': 'quantidade_lag_52_calc'})

        # Merge with the submission dataset based on pdv, produto, and semana
        df_sub_preparado = df_sub_preparado.merge(
            semana_ano_anterior_stats,
            on=['pdv', 'produto', 'semana'],
            how='left'
        )

        # If the calculated lag_52 is available, use it, otherwise use the one from ultimo_registro_historico or 0
        df_sub_preparado['quantidade_lag_52'] = df_sub_preparado['quantidade_lag_52_calc'].fillna(
            df_sub_preparado['quantidade_lag_52'].fillna(0) # Fill with merged historical lag_52 if exists, else 0
        )

        # Dropar the temporary column
        df_sub_preparado = df_sub_preparado.drop(columns=['quantidade_lag_52_calc'])
    else:
         # If 'semana' is not available or stats couldn't be calculated, ensure lag_52 is filled with 0
         if 'quantidade_lag_52' not in df_sub_preparado.columns:
              df_sub_preparado['quantidade_lag_52'] = 0
         else:
              df_sub_preparado['quantidade_lag_52'] = df_sub_preparado['quantidade_lag_52'].fillna(0)


    # Preencher NaN remaining in the merged feature columns with 0
    cols_to_fill_zero = [col for col in actual_features_to_merge if col not in ['pdv', 'produto']]
    for col in cols_to_fill_zero:
         if col in df_sub_preparado.columns: # Check if column exists after merge
              df_sub_preparado[col] = df_sub_preparado[col].fillna(0)
         else:
              # If a feature column wasn't in historical data for merging, add it with 0
              df_sub_preparado[col] = 0


    # Define the expected features for the model (must match training features)
    expected_features = ['pdv', 'produto', 'ano', 'mes', 'semana',
                         'quantidade_lag_1', 'quantidade_lag_2', 'quantidade_lag_3',
                         'quantidade_lag_4', 'quantidade_lag_52',
                         'media_movel_4', 'media_movel_8', 'media_movel_13']

    # Ensure the prepared dataframe has all expected features, adding them with 0 if missing
    for feature in expected_features:
        if feature not in df_sub_preparado.columns:
            df_sub_preparado[feature] = 0

    # Select and reorder columns to match the expected features
    df_sub_preparado = df_sub_preparado[expected_features]

    print("\nColumns of the prepared submission dataset:\n", df_sub_preparado.columns.tolist())
    print("\nShape of the prepared submission dataset:", df_sub_preparado.shape)


    return df_sub_preparado

In [6]:
def remove_outliers(df):
    print("Antes:", df.shape)
    first_quartile = df['quantidade'].describe()['25%']
    third_quartile = df['quantidade'].describe()['75%']

    iqr = third_quartile - first_quartile

    df_sem_outliers= df[(df['quantidade'] > (first_quartile - 1.5 * iqr)) &
            (df['quantidade'] < (third_quartile + 1.5 * iqr))].copy()

    # Valores negativos em quantidade são removidos
    df_sem_outliers = df_sem_outliers[df_sem_outliers['quantidade'] >= 0]
    print("Depois:", df_sem_outliers.shape)
    return df_sem_outliers

In [7]:
def criar_dataset_teste_submissao(df_treino):
    """
    Criar dataset de teste para submissão com semanas 1 a 5 de janeiro
    Retorna 5 linhas com pdv e produto diferentes.
    """
    # Identificar combinações únicas de pdv e produto
    combinacoes_unicas = df_treino[['pdv', 'produto']].drop_duplicates()

    # Selecionar as primeiras 5 combinações únicas
    combinacoes_selecionadas = combinacoes_unicas.head(5)

    semanas_janeiro = [1, 2, 3, 4, 5]

    dados_teste = []
    # Para cada uma das 5 combinações selecionadas, criar uma linha com uma semana diferente de 1 a 5
    for i, combo in enumerate(combinacoes_selecionadas.iterrows()):
        pdv_id = combo[1]['pdv'] # Access the row data using index 1
        produto_id = combo[1]['produto'] # Access the row data using index 1

        dados_teste.append({
            'pdv': pdv_id,
            'produto': produto_id,
            'semana': semanas_janeiro[i] # Use the enumerate index 'i' to assign weeks 1-5 sequentially
        })


    # Criar DataFrame
    df_teste = pd.DataFrame(dados_teste)

    # Garantir que as colunas são do tipo int
    df_teste['semana'] = df_teste['semana'].astype(int)

    return df_teste

In [8]:
def treinar_e_avaliar_modelo(df_treino):
    """
    Treinar modelo com dados de treino
    """
    features = ['pdv', 'produto', 'ano', 'mes', 'semana',  'quantidade',
                'quantidade_lag_1', 'quantidade_lag_2', 'quantidade_lag_3',
                'quantidade_lag_4', 'quantidade_lag_52',
                'media_movel_4', 'media_movel_8', 'media_movel_13']

    # Separate features (X) and target (y)
    # Exclude 'data' as it's not needed for the model and 'quantidade' as it's the target
    df_treino = df_treino[features]
    X = df_treino.drop(['quantidade'], axis=1)
    y = df_treino['quantidade']

    # Split the data into training and testing sets
    # Using a test size of 30% and random_state for reproducibility
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    print("Shape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)

    print("Treinando modelo")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    print("Modelo treinado!")
    y_pred = model.predict(X_test)
    wmape_score = wmape(y_test, y_pred)
    print(f"WMAPE: {wmape_score:.4f} ({wmape_score*100:.2f}%)")
    return model

In [9]:
# Função para calcular WMAPE
def wmape(y_true, y_pred):
    """
    Weighted Mean Absolute Percentage Error
    WMAPE = Σ|y_true - y_pred| / Σ|y_true|
    """
    print('Aplicando WMAPE')
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [14]:
path='/content/drive/MyDrive/HACKA/'

# Ler os arquivos parquet com  pyarrow
- cada parte do arquivo parquet foi lida individualmente
- depois foram concatenadas
- a quantidade de linhas e colunas ficou igual a leitura com duckdb

In [15]:
# Get the directory of the current file
directory = os.path.dirname(path)

# List all parquet files in the directory that are part of the dataset
all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.startswith('part-') and f.endswith('.parquet')]

#print("Arquivos Parquet encontrados:")
#for file in all_files:
    #print(file)
print(f'Lendo e concatenando registros...')

tables = []
for file in all_files:
    try:
        # Read each parquet file into a PyArrow table
        table = pq.read_table(file)
        tables.append(table)
        #print("\nTables Pyarrow:\n", tables)
    except Exception as e:
        print(f"Error reading file {file}: {e}")

if tables:
    # Concatenate the PyArrow tables
    # Use the promote parameter to handle schema differences by promoting to a common type
    combined_table = pa.concat_tables(tables, promote=True)

    # Convert the combined PyArrow Table to a Pandas DataFrame
    df_pyarrow_combined = combined_table.to_pandas()

    print("\nDataset pandas lido e concatenado pelo PyArrow (todas as colunas):")
    print("Colunas:", df_pyarrow_combined.columns.tolist())
    print(df_pyarrow_combined.shape)
else:
    print("No tables were read.")

Lendo e concatenando registros...

Dataset pandas lido e concatenado pelo PyArrow (todas as colunas):
Colunas: ['pdv', 'premise', 'categoria_pdv', 'zipcode', 'produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante', 'internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']
(6582209, 23)


In [16]:
df_pyarrow_combined.columns

Index(['pdv', 'premise', 'categoria_pdv', 'zipcode', 'produto', 'categoria',
       'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante',
       'internal_store_id', 'internal_product_id', 'distributor_id',
       'transaction_date', 'reference_date', 'quantity', 'gross_value',
       'net_value', 'gross_profit', 'discount', 'taxes'],
      dtype='object')

In [17]:
df_pyarrow_combined.head()

,pdv,premise,categoria_pdv,zipcode,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,2204965430669363375,On Premise,Mexican Rest,30741.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,5211957289528622910,On Premise,Hotel/Motel,80011.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,9024493554530757353,Off Premise,Convenience,80751.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,8659197371382902429,On Premise,Restaurant,80439.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,1400854873763881130,On Premise,Restaurant,30093.0,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


# Leitura dos arquivos parquet com duckdb
- esta leitura foi descontinuada pois o pyarrow ja fez isso.

# Coluna e tipos esperados no dataset de teste e submissão
- coluna semana no ano (1 a 52)
- pdv (número inteiro): código do ponto de venda
- produto (número inteiro): código do SKU
- quantidade (número inteiro): previsão de vendas

In [18]:
df = df_pyarrow_combined.copy()

# Estatistica Descritiva

In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
zipcode,14419.0,62676.248769,24052.659458,8107.000000,30341.000000,80202.000000,80535.000000,90920.000000
quantity,6560698.0,8.125893,80.493873,-1530.000000,1.000000,2.000000,4.000000,94230.000000
gross_value,6560698.0,122.751233,866.426496,-42672.898438,28.035000,42.099998,92.000000,604173.917786
net_value,6560698.0,119.123296,865.175682,-39848.000046,27.110323,40.770485,87.879002,604173.917786
gross_profit,6560698.0,21.930921,232.646984,-274396.000000,7.140242,10.510241,21.732000,274416.000000
discount,6560698.0,27.666835,384.762433,-13096.799805,0.000000,2.300000,8.700000,240082.965628
taxes,6560698.0,3.627938,11.309847,-4099.408691,0.180000,0.540000,3.039677,2073.237549


# Tipo de cada coluna

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6582209 entries, 0 to 6582208
Data columns (total 23 columns):
 #   Column               Dtype  
---  ------               -----  
 0   pdv                  object 
 1   premise              object 
 2   categoria_pdv        object 
 3   zipcode              float64
 4   produto              object 
 5   categoria            object 
 6   descricao            object 
 7   tipos                object 
 8   label                object 
 9   subcategoria         object 
 10  marca                object 
 11  fabricante           object 
 12  internal_store_id    object 
 13  internal_product_id  object 
 14  distributor_id       object 
 15  transaction_date     object 
 16  reference_date       object 
 17  quantity             float64
 18  gross_value          float64
 19  net_value            float64
 20  gross_profit         float64
 21  discount             float64
 22  taxes                float64
dtypes: float64(7), object(16)
memory

# Percentual de dados ausentes por coluna

In [21]:
missing_values_table(df)

Seu dataframe tem 23 colunas.
Há 23 colunas que possuem valores ausentes.


,Missing Values,% of Total Values
label,6576590,99.9
subcategoria,6575149,99.9
descricao,6575117,99.9
marca,6575117,99.9
tipos,6575117,99.9
categoria,6575117,99.9
produto,6575117,99.9
fabricante,6575117,99.9
pdv,6567790,99.8
premise,6567790,99.8


# Colunas pdv e produto apresentam 99% de dados ausentes.
# Serao substituidas por
- internal_store_id
- internal_product_id

In [22]:
colunas = ['internal_store_id',  'pdv',  'internal_product_id', 'produto', 'transaction_date','quantity']

In [ ]:
missing_values_table(df[colunas])

# Ano  remover as linhas ausentes em quantity as linhas de outras colunas também foram removidas.

In [23]:
df = df.dropna(subset=["quantity"]).copy()
missing_values_table(df[colunas])

Seu dataframe tem 6 colunas.
Há 2 colunas que possuem valores ausentes.


,Missing Values,% of Total Values
pdv,6560698,100.0
produto,6560698,100.0


In [24]:
df.columns

Index(['pdv', 'premise', 'categoria_pdv', 'zipcode', 'produto', 'categoria',
       'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante',
       'internal_store_id', 'internal_product_id', 'distributor_id',
       'transaction_date', 'reference_date', 'quantity', 'gross_value',
       'net_value', 'gross_profit', 'discount', 'taxes'],
      dtype='object')

In [27]:
df2 = df[['internal_store_id', 'internal_product_id', 'transaction_date', 'quantity']].copy()

# Coluna são renomeadas

In [28]:
df2.rename(columns={'internal_store_id': 'pdv', 'internal_product_id': 'produto', 'quantity': 'quantidade', 'transaction_date': 'data'}, inplace=True)

In [30]:
df2.isna().sum()

,0
pdv,0
produto,0
data,0
quantidade,0
semana,0


# Extrair o numero da semana no ano a partir da data de transacao

In [31]:
df2['data'] = pd.to_datetime(df2['data'])
df2['semana'] = df2['data'].dt.isocalendar().week

In [32]:
df2.columns

Index(['pdv', 'produto', 'data', 'quantidade', 'semana'], dtype='object')

In [33]:
df2

,pdv,produto,data,quantidade,semana
21511,7384367747233276219,328903483604537190,2022-07-13,1.0,28
21512,3536908514005606262,5418855670645487653,2022-03-21,6.0,12
21513,3138231730993449825,1087005562675741887,2022-09-06,3.0,36
21514,3681167389484217654,1401422983880045188,2022-09-11,129.0,36
21515,7762413312337359369,6614994347738381720,2022-02-18,1.0,7
...,...,...,...,...,...
6582204,1439579822593789678,2523145330235258825,2022-12-06,3.0,49
6582205,2357589266242432199,3148628429396585683,2022-05-20,1.0,20
6582206,7245187752757348055,6668439675743950745,2022-08-17,1.0,33
6582207,8952609383990245450,3894706280449257667,2022-02-03,4.0,5


In [34]:
missing_values_table(df2)

Seu dataframe tem 5 colunas.
Há 0 colunas que possuem valores ausentes.


,Missing Values,% of Total Values


In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6560698 entries, 21511 to 6582208
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   pdv         object        
 1   produto     object        
 2   data        datetime64[ns]
 3   quantidade  float64       
 4   semana      UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(1), object(2)
memory usage: 281.6+ MB


In [36]:
df2

,pdv,produto,data,quantidade,semana
21511,7384367747233276219,328903483604537190,2022-07-13,1.0,28
21512,3536908514005606262,5418855670645487653,2022-03-21,6.0,12
21513,3138231730993449825,1087005562675741887,2022-09-06,3.0,36
21514,3681167389484217654,1401422983880045188,2022-09-11,129.0,36
21515,7762413312337359369,6614994347738381720,2022-02-18,1.0,7
...,...,...,...,...,...
6582204,1439579822593789678,2523145330235258825,2022-12-06,3.0,49
6582205,2357589266242432199,3148628429396585683,2022-05-20,1.0,20
6582206,7245187752757348055,6668439675743950745,2022-08-17,1.0,33
6582207,8952609383990245450,3894706280449257667,2022-02-03,4.0,5


In [37]:
df2.nunique()

,0
pdv,15086
produto,7092
data,365
quantidade,16449
semana,52


# Coluna quantidade tem valores negativos
# Outlier certamente.

In [38]:
df2.describe()

,data,quantidade,semana
count,6560698,6.560698e+06,6560698.0
mean,2022-07-12 12:28:13.641744384,8.125893e+00,27.934699
min,2022-01-01 00:00:00,-1.530000e+03,1.0
25%,2022-04-20 00:00:00,1.000000e+00,16.0
50%,2022-07-19 00:00:00,2.000000e+00,29.0
75%,2022-10-03 00:00:00,4.000000e+00,40.0
max,2022-12-31 00:00:00,9.423000e+04,52.0
std,NaN,8.049387e+01,14.337151


In [39]:
df_sem_outliers = remove_outliers(df2)

Antes: (6560698, 5)
Depois: (5668967, 5)


# Extrair entre 10% e 100% dos dados durante teste.

In [40]:
df_100 = df_sem_outliers.sample(frac=1, random_state=42) # 100%
df_100.shape

(5668967, 5)

In [41]:
df_100.head()

,pdv,produto,data,quantidade,semana
22412,3462134966385778158,1438376043637575068,2022-11-21,1.0,47
2261493,7258564421045379389,4353552881410365573,2022-06-03,1.0,22
577148,320309252382737243,8824636260730784316,2022-06-13,3.0,24
1277935,8887613577044431590,4767947340064453366,2022-01-12,3.0,2
5214828,3757377484856476522,1288112997726060694,2022-04-04,1.0,14


In [42]:
df_treino = criar_features_para_treinamento(df_100)

Features: ['pdv', 'produto', 'data', 'quantidade', 'semana', 'ano', 'mes', 'quantidade_lag_1', 'quantidade_lag_2', 'quantidade_lag_3', 'quantidade_lag_4', 'quantidade_lag_52', 'media_movel_4', 'media_movel_8', 'media_movel_13']


In [43]:
df_treino.isna().sum()

,0
pdv,0
produto,0
data,0
quantidade,0
semana,0
ano,0
mes,0
quantidade_lag_1,0
quantidade_lag_2,0
quantidade_lag_3,0


In [44]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5668967 entries, 22412 to 1983033
Data columns (total 15 columns):
 #   Column             Dtype         
---  ------             -----         
 0   pdv                object        
 1   produto            object        
 2   data               datetime64[ns]
 3   quantidade         float32       
 4   semana             Int64         
 5   ano                Int64         
 6   mes                Int64         
 7   quantidade_lag_1   float32       
 8   quantidade_lag_2   float32       
 9   quantidade_lag_3   float32       
 10  quantidade_lag_4   float32       
 11  quantidade_lag_52  float32       
 12  media_movel_4      float32       
 13  media_movel_8      float32       
 14  media_movel_13     float32       
dtypes: Int64(3), datetime64[ns](1), float32(9), object(2)
memory usage: 513.6+ MB


In [45]:
df_sub = criar_dataset_teste_submissao(df_treino)

In [46]:
df_sub

,pdv,produto,semana
0,3462134966385778158,1438376043637575068,1
1,7258564421045379389,4353552881410365573,2
2,320309252382737243,8824636260730784316,3
3,8887613577044431590,4767947340064453366,4
4,3757377484856476522,1288112997726060694,5


In [47]:
df_sub_preparado = criar_features_para_submissao(df_sub, df_treino)


Columns of the prepared submission dataset:
 ['pdv', 'produto', 'ano', 'mes', 'semana', 'quantidade_lag_1', 'quantidade_lag_2', 'quantidade_lag_3', 'quantidade_lag_4', 'quantidade_lag_52', 'media_movel_4', 'media_movel_8', 'media_movel_13']

Shape of the prepared submission dataset: (5, 13)


In [ ]:
model = treinar_e_avaliar_modelo(df_treino)

Shape of X_train: (3968276, 13)
Shape of X_test: (1700691, 13)
Shape of y_train: (3968276,)
Shape of y_test: (1700691,)
Treinando modelo


In [ ]:
df_sub['quantidade'] = model.predict(df_sub_preparado)
df_sub

In [ ]:
# Round the predicted quantities to the nearest integer before converting to int type
df_sub['quantidade'] = df_sub['quantidade'].round().astype(int)

In [ ]:
df_sub

In [ ]:
# Use the df_sub with the rounded integer 'quantidade'
df_envio = df_sub[['pdv', 'produto', 'semana', 'quantidade']].copy()
df_envio['produto'] = df_envio['produto'].astype(int)
df_envio['pdv'] = df_envio['pdv'].astype(int)
df_envio['semana'] = df_envio['semana'].astype(int)
df_envio.columns = ['semana','pdv', 'produto', 'quantidade']

df_envio.to_csv('submission.csv', sep=';', encoding='utf-8', index=False)
display(df_envio)

In [ ]:
df_envio.info()